In [11]:
import time
import matplotlib.pyplot as plt
import numpy as np
import math as mt
import seaborn as sns
from tqdm import tqdm
import pandas as pd
from konlpy.tag import Mecab
from hangul_utils import split_syllables, join_jamos
import re
#from pykospacing import Spacing
#if you use window, import eungeon instand of konlpy
#from eunjeon import Mecab

mec = Mecab()

##높임말 리스트. 다음에 해당하면 높임말로 판단한다.
H_LIST = ['니다', '니까', '요', '시오', '죠']

##높임말로 변환할 때 중성끼리 합치는 경우가 있다. 그러한 경우 합치기 위한 사전이다. 
con_dict = [
    
    ['ㅏㅣ','ㅐ'], ['ㅑㅣ','ㅒ'], ['ㅓㅣ','ㅔ'],
    ['ㅕㅣ','ㅖ'], ['ㅗㅣ','ㅚ'], ['ㅗㅐ','ㅙ'],
    ['ㅜㅓ','ㅝ'], ['ㅜㅔ','ㅞ'], ['ㅡㅣ','ㅢ'],
    ['ㅣㅏ','ㅑ'], ['ㅣㅓ','ㅕ'], ['ㅣㅗ','ㅛ'],
    ['ㅣㅜ','ㅠ'], ['ㅡㅓ','ㅓ']
    
]

####high -> low Dictionary####

##EF(종결어미): 형태소 분류 시 종결어미가 단독으로 분석 되는 경우
EF_ONLY = [
    
    [['ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['-special']], #ex)'습니다'-> '다'
    [['ㄴ','ㅏ','ㅇ','ㅛ'],['ㄴ','ㅏ']],
    [['ㄷ','ㅐ','ㅇ','ㅛ'],['ㄷ','ㅐ']],
    [['ㅇ','ㅔ','ㅇ','ㅛ'],['ㅇ','ㅑ']],
    [['ㄴ','ㅔ','ㅇ','ㅛ'],['ㄷ','ㅏ']],
    [['ㄴ','ㅡ','ㄴ','ㄷ','ㅔ','ㅇ','ㅛ'],['ㄴ','ㅡ','ㄴ','ㄷ','ㅔ']],
    [['ㅈ','ㅛ'],['ㅈ','ㅣ']],
    [['ㅇ','ㅏ','ㅇ','ㅛ'],['ㅇ','ㅏ']],
    [['ㅇ','ㅓ','ㅇ','ㅛ'],['ㅇ','ㅓ']],
    [['ㅇ','ㅡ','ㅅ','ㅔ','ㅇ','ㅛ'],['ㅇ','ㅓ']],
    [['ㄱ','ㅓ','ㄷ','ㅡ','ㄴ','ㅇ','ㅛ'],['ㄱ','ㅓ','ㄷ','ㅡ','ㄴ']],
    #[['ㅎ','ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄲ','ㅏ'],['ㅎ','ㅇ','ㅏ']],
    [['ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄲ','ㅏ'],['ㅇ','ㅓ']],
    [['ㄴ','ㅡ','ㄴ','ㄷ','ㅔ','ㅇ','ㅛ'],['ㄴ','ㅡ','ㄴ','ㄷ','ㅔ']],
    [['ㅇ','ㅛ'],['']],
    
]

#EP(선어말어미)+EF(종결어미)로 형태소가 분석될 경우
EP_EF = [
    
    [['ㅅ','ㅔ','ㅇ','ㅛ'],['special']]
    #-시오 family
    ,[['ㅡ','ㅅ','ㅣ','ㅂ','ㅅ','ㅣ','ㅇ','ㅗ'],['ㅓ']]
    ,[['ㅅ','ㅣ','ㅂ','ㅅ','ㅣ','ㅇ','ㅗ'],['ㄹ', 'ㅏ']],
    [['ㅅ','ㅣ','ㅂ','ㄴ','ㅣ','ㄲ','ㅏ'],['ㄴ','ㅣ']],
    [['ㅅ','ㅣ','ㄹ','ㄹ','ㅐ','ㅇ','ㅛ'],['ㄹ','ㄹ','ㅐ']],
]

#VCP+EF
VCP_EF = [
    
    [['ㅇ','ㅣ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㅇ','ㅣ','ㄷ','ㅏ']],
    [['ㅇ','ㅣ','ㅂ','ㄴ','ㅣ','ㄲ','ㅏ'],['ㅇ','ㅣ','ㅇ','ㅑ']],
    [['ㅈ','ㅛ'],['ㅇ','ㅑ']],
    [['ㅇ','ㅣ','ㅇ','ㅔ','ㅇ','ㅛ'],['ㅇ','ㅣ','ㅇ','ㅑ']],
    [['ㅇ','ㅖ','ㅇ','ㅛ'],['ㅇ','ㅑ']],
    [['ㄱ','ㅓ','ㅇ','ㅛ'],['ㅇ','ㅑ']],
    [['ㄹ','ㅏ','ㄷ','ㅓ','ㄴ','ㄷ','ㅔ','ㅇ','ㅛ'],['ㄹ','ㅏ','ㄷ','ㅓ','ㄴ','ㄷ','ㅔ']],
    
]

#XSA+EF
XSA_EF = [
    [['ㅎ','ㅏ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㅎ','ㅏ','ㄷ','ㅏ']],
    [['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㄷ','ㅏ']],
    [['ㅇ','ㅛ'],['']]
]

#XSV+EF
XSV_EF = [
    [['ㅎ','ㅏ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㅎ','ㅐ']],
    [['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㄷ','ㅏ']],
    [['ㅎ','ㅐ','ㅇ','ㅛ'],['ㅎ','ㅏ','ㅈ','ㅏ']],
]

#alpha+EF: 위의 경우를 제외한 나머지 경우를 의미한다. 
A_EF = [
    [['ㄱ','ㅓ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㄷ','ㅏ']],
    [['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㄴ','ㄷ','ㅏ']],
    [['ㄱ','ㅖ','ㅅ','ㅔ','ㅇ','ㅛ'],['ㅇ','ㅣ','ㅆ','ㅇ','ㅓ']],
    [['ㅂ','ㄴ','ㅣ','ㄲ','ㅏ'],['ㄴ','ㅏ']],
    [['ㅇ','ㅛ'],['']]
    
    
]

##높임표현이 어말어미가 아닌 경우##
# - 기존의 변환 방식이었다. 이제는 쓰지는 않지만, 코드의 오류가 발생할 가능성이 있기에 지우지 않았다. 따로 참고할 필요 없음.
EF = [
    [['ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㄷ','ㅏ']],
    [[' ','ㅈ','ㅓ','ㄴ',' '],['ㄴ','ㅏ','_','ㄴ','ㅡ','ㄴ']],
    [['ㅇ','ㅓ','_','ㅇ','ㅛ','_'],['ㄷ','ㅏ','_']],
    [['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㄴ','ㄷ','ㅏ']],
    [['ㅇ','ㅔ','ㅇ','ㅛ'],['ㅇ','ㅑ']],
    [['ㅇ','ㅛ'],['']]
    #,[['ㅅ','ㅔ'],['ㅝ'],['ㅘ']]
]

NP = [
    
    [['ㅈ','ㅓ','ㄴ'],['special-']],
    [['ㅈ','ㅓ'],['ㄴ','ㅏ']]
    
]

# 보조사
JX = [
    [['ㅇ','ㅣ','ㅇ','ㅛ'],['ㅇ','ㅣ','ㅇ','ㅑ']],
    [['ㅇ','ㅛ'],['']]
]

VX = [
    
    [['ㅈ','ㅜ'],['ㅈ','ㅜ'],['ㅈ','ㅝ']]
    
]

VV = [
    
    [['ㅇ','ㅗ'],['ㅇ','ㅗ'],['ㅇ','ㅘ']],
    [['ㅈ','ㅜ'],['ㅈ','ㅜ'],['ㅈ','ㅝ']],
    [['ㅎ','ㅏ'],['ㅎ','ㅏ'],['ㅎ','ㅐ']]
    
]

XR = [
    [['ㅈ','ㅚ','ㅅ','ㅗ','ㅇ'],['ㅁ','ㅣ','ㅇ','ㅏ','ㄴ'] ]
]

NNG = [
    [['ㅈ','ㅏ','ㅇ','ㅛ'],['ㅈ','ㅏ']],
    [['ㄱ','ㅏ','ㅁ','ㅅ','ㅏ'],['ㅈ','ㅏ']],
]

NNP =[
    [[],[]]
]

EXC_deal_1 = [
    
    ['ㅅㅔㅇㅛ', ['ㅏ','ㅣ'],['ㅜ','ㅓ'],['ㅡ','ㅓ']],
]

EXC_deal_2 = [
    
    ['ㅈㅓㄴ','ㅈㅓㄴ','ㄴㅏㄴ'],
]

EXC_deal_3 = [
    
    ['ㅅㅡㅂㄴㅣㄷㅏ',['ㅆ','ㄷㅏ'],'ㄴㅡㄴㄷㅏ'],
    
]

EXC_tags = [
    
    'EP',
    'EF'
    
]

UNKNOWN = [
    [['ㄷ','ㅡ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㄷ','ㅡ','ㄴ','ㄷ','ㅏ']],
]

####End of high -> low Dictionary####


#### low -> high Dictionary####
## 사전의 순서가 중요하다. 순서를 임의로 바꾸지 말 것
## EF solo. EF 앞에 형태소가 결합형이 아닌 홀로 분석 될 경우 ex) /EP, /EF

EF_ONLY_4S = [
    # -다면서 /EF
    [['ㄷ','ㅏ','ㅁ','ㅕ','ㄴ','ㅅ','ㅓ'],['ㄷ','ㅏ','ㅁ','ㅕ','ㄴ','ㅅ','ㅓ','ㅇ','ㅛ']],
    # -고 /EF
    [['ㄹ','ㅏ','ㄱ','ㅗ'],['ㄹ','ㅏ','ㄱ','ㅗ','ㅇ','ㅛ']], ##-라고-> 라고요
    [['ㄷ','ㅓ','ㄹ','ㅏ','ㄱ','ㅗ'],['ㄷ','ㅓ','ㄹ','ㅏ','ㄱ','ㅗ','ㅇ','ㅛ']],
    [['ㄷ','ㅏ','ㄱ','ㅗ'],['ㄷ','ㅏ','ㄱ','ㅗ','ㅇ','ㅛ']],
    
    [['ㄷ','ㅏ'],['ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['special']],
    [['ㅇ','ㅓ','ㄹ','ㅏ'],['ㅇ','ㅡ','ㅅ','ㅔ','ㅇ','ㅛ']], #add jh_1115
    [['ㅇ','ㅓ'],['ㅇ','ㅓ','ㅇ','ㅛ']],
    #'-라' /EF
    [['ㄷ','ㅓ','ㄹ','ㅏ'],['ㄷ','ㅓ','ㄴ','ㄷ','ㅔ','ㅇ','ㅛ']],
    [['ㅇ','ㅏ','ㄹ','ㅏ'],['ㅇ','ㅏ','ㅇ','ㅛ']],
    [['ㄹ','ㅏ'],['ㅅ','ㅔ','ㅇ','ㅛ']],
    
    # -데 /EF
    [['ㅇ','ㅡ','ㄴ','ㄷ','ㅔ'],['ㅇ','ㅡ','ㄴ','ㄷ','ㅔ','ㅇ','ㅛ']],
    [['ㄴ','ㅡ','ㄴ','ㄷ','ㅔ'],['ㄴ','ㅡ','ㄴ','ㄷ','ㅔ','ㅇ','ㅛ']],
    [['ㄷ','ㅓ','ㄴ','ㄷ','ㅔ'],['ㄷ','ㅓ','ㄴ','ㄷ','ㅔ','ㅇ','ㅛ']],
    # -걸 /EF
    [['ㄴ','ㅡ','ㄴ','ㄱ','ㅓ','ㄹ'],['ㄴ','ㅡ','ㄴ','ㄱ','ㅓ','ㄹ','ㅇ','ㅛ']],
    
    [['ㅈ','ㅏ','ㄶ','ㅇ','ㅏ'],['ㅈ','ㅏ','ㄶ','ㅇ','ㅏ','ㅇ','ㅛ']], ## -잖아 -> -잖아요
    
    #-오 /EF
    [['ㅇ','ㅗ'],['ㅅ','ㅔ','ㅇ','ㅛ']],
    #-자 /EF
    [['ㅈ','ㅏ'],['ㅅ','ㅔ','ㅇ','ㅛ']],
    #-래 /EF
    [['ㅇ''ㅡ','ㄹ','ㄹ','ㅐ'],['ㅇ''ㅡ','ㄹ','ㄹ','ㅐ','ㅇ','ㅛ']],
    [['ㄹ','ㅐ'],['ㄹ','ㅐ','ㅇ','ㅛ']],
    #-니 /EF
    [['ㄴ','ㅣ'],['ㄴ','ㅏ','ㅇ','ㅛ']],
    
    # -거든 /EF
    [['ㄱ','ㅓ','ㄷ','ㅡ','ㄴ'],['ㄱ','ㅓ','ㄷ','ㅡ','ㄴ','ㅇ','ㅛ']],
    
    #-마 /EF
    [['ㅁ','ㅏ'],['ㅁ','ㅏ','ㅇ','ㅛ']],
    
    #-아 /EF
    [['ㅇ','ㅏ'],['ㅇ','ㅏ','ㅅ','ㅔ','ㅇ','ㅛ']],
    #-지/EF
    [['ㅈ','ㅣ'],['ㅈ','ㅛ']],
    
    [['ㅏ'],['ㅏ','ㅅ','ㅔ','ㅇ','ㅛ']],
    
    
    
]
## EF 결합형
## EF 앞에 형태소가 결합형인 경우 앞의 형태소도 고려해서 처리해야 한다. ex) /EP, /EF
EF_ONLY_4C = [
    
    [['ㄷ','ㅏ'],['ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']],
    [['ㅇ','ㅓ'],['ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']],
    [['ㅇ','ㅓ','ㄸ','ㅐ'],['ㅇ','ㅓ','ㄸ','ㅐ','ㅇ','ㅛ']],
    [['ㄴ','ㅡ','ㄴ','ㄷ','ㅔ'],['ㄴ','ㅡ','ㄴ','ㄷ','ㅔ','ㅇ','ㅛ']],
    
    # -걸 /EF
    [['ㄴ','ㅡ','ㄴ','ㄱ','ㅓ','ㄹ'],['ㄴ','ㅡ','ㄴ','ㄱ','ㅓ','ㄹ','ㅇ','ㅛ']],
    [['ㄴ','ㅣ'],['-special']],
    # -니 /EF
    [['ㄴ','ㅣ'],['ㅇ','ㅓ','ㅇ','ㅛ']],
    # -거든 /EF
    [['ㄱ','ㅓ','ㄷ','ㅡ','ㄴ'],['ㄱ','ㅓ','ㄷ','ㅡ','ㄴ','ㅇ','ㅛ']],
    [['ㅈ','ㅣ'],['ㅈ','ㅛ']],
    
]

#/VCP+EF
VCP_EF_4 = [
    
    [['ㄱ','ㅓ','ㄹ'],['ㄷ','ㅔ','ㅇ','ㅛ']],
    [['ㅇ','ㅑ'],['ㅇ','ㅔ','ㅇ','ㅛ']],
    [['ㄹ','ㅐ'],['ㄹ','ㅐ','ㅇ','ㅛ']],
    [['ㄴ','ㅣ'],['ㅇ','ㅔ','ㅇ','ㅛ']], #add jh_1116
    [['ㄷ','ㅔ'],['ㄷ','ㅔ','ㅇ','ㅛ']],
    
]

#/VA+EF added code_JH
VA_EF_4 = [
    [['ㅇ','ㅓ','ㄸ','ㅐ'],['ㅇ','ㅓ','ㄸ','ㅐ','ㅇ','ㅛ']],
    [['ㄴ','ㅣ'],['-special']],
    [['ㅕ'],['ㅕ','ㅇ','ㅛ']],
]

## /VCP, /EF
EF_AFTER_VCP_4 = [
    
    [['ㄷ','ㅏ'],['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']],
    #-오 /EF
    [['ㅇ','ㅗ'],['ㅅ','ㅔ','ㅇ','ㅛ']],
    [['ㄴ','ㅣ'],['ㅇ','ㅔ','ㅇ','ㅛ']], #add jh_1116
    
]

EF_AFTER_XSA_4 = [
    
    [['ㄷ','ㅏ'],['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']],
    
]

VV_EF_4 = [
    [['ㄱ','ㅓ','ㄹ'],['ㄱ','ㅓ','ㄹ','ㅇ','ㅛ']],
    [['ㅎ','ㅐ','ㄹ','ㅏ'],['ㅎ','ㅏ','ㅅ','ㅔ','ㅇ','ㅛ']],
    [['ㅂ','ㅘ','ㄹ','ㅏ'],['ㅂ','ㅗ','ㅅ','ㅔ','ㅇ','ㅛ']],
    [['ㅂ','ㅘ'],['ㅂ','ㅘ','ㅇ','ㅛ']],
    [['ㅎ','ㅏ','ㄹ','ㄹ','ㅐ'],['ㅎ','ㅏ','ㄹ','ㄹ','ㅐ','ㅇ','ㅛ']],
    [['ㅈ','ㅜ','ㄹ','ㄹ','ㅐ'],['ㅈ','ㅜ','ㄹ','ㄹ','ㅐ','ㅇ','ㅛ']], #add jh_1115
    [['ㄷ','ㅙ'],['ㄷ','ㅙ','ㅇ','ㅛ']],
    [['ㄴ','ㅣ'],['-special']],
    [['ㄷ','ㅡ','ㄹ','ㄹ','ㅕ'],['ㄷ','ㅡ','ㄹ','ㄹ','ㅕ','ㅇ','ㅛ']],
    [['ㄷ','ㅏ','ㅁ','ㅕ','ㄴ','ㅅ','ㅓ'],['ㄷ','ㅏ','ㅁ','ㅕ','ㄴ','ㅅ','ㅓ','ㅇ','ㅛ']],
    [['ㄹ','ㅐ'],['ㄹ','ㅐ','ㅇ','ㅛ']],
    [['ㅕ'],['ㅕ','ㅇ','ㅛ']],
    [['ㅊ','ㅝ'],['ㅊ','ㅝ','ㅇ','ㅛ']],
    [['ㄴ','ㄷ','ㅏ'],['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']],
    [['ㅏ'],['ㅏ','ㅇ','ㅛ']],
    
]

##added code_JH
XSV_EF_4 = [
    [['ㅎ','ㅐ'],['ㅎ','ㅐ','ㅇ','ㅛ']],
    [['ㄹ','ㄲ','ㅏ'],['ㄹ','ㄲ','ㅏ','ㅇ','ㅛ']],
]
XSA_EF_4 = [
    [['ㅎ','ㅏ','ㄴ','ㄷ','ㅔ'],['ㅎ','ㅏ','ㄴ','ㄷ','ㅔ','ㅇ','ㅛ']],
    [['ㅎ','ㅐ'],['ㅎ','ㅐ','ㅇ','ㅛ']],
]

VX_EF_4 = [
    [['ㅎ','ㅐ'],['ㅎ','ㅐ','ㅇ','ㅛ']],
    [['ㄹ','ㅏ'],['ㅇ','ㅛ']],
    [['ㅂ','ㅘ','ㄹ','ㅏ'],['ㅂ','ㅗ','ㅅ','ㅔ','ㅇ','ㅛ']],
    [['ㅂ','ㅘ'],['ㅂ','ㅘ','ㅇ','ㅛ']],
    [['ㅈ','ㅜ','ㄹ','ㄹ','ㅐ'],['ㅈ','ㅜ','ㄹ','ㄹ','ㅐ','ㅇ','ㅛ']], #add jh_1115
    [['ㅁ','ㅏ'],['ㅁ','ㅏ','ㅇ','ㅛ']],
    [['ㄹ','ㄱ','ㅔ'],['ㄹ','ㄱ','ㅔ','ㅇ','ㅛ']],
    [['ㅝ'],['ㅝ','ㅇ','ㅛ']],
    
]

NNB_JKO_4 = [
    [['ㄱ','ㅓ','ㄹ'],['specialx']],
]

EC_4 = [
    [['ㄷ','ㅔ'],['specialx']],
    #[['ㄷ','ㅔ'],['ㄷ','ㅔ','ㅇ','ㅛ']],
    [['ㄱ','ㅔ'],['specialx']],
    #[['ㄱ','ㅔ'],['ㄱ','ㅔ','ㅇ','ㅛ']],
]

VV_4 = [
    
    #[['ㄷ','ㅚ'],['ㄷ','ㅚ','ㅇ','ㅛ']],
]

EXC_4_deal_1 = [
    
    ['ㄴㅣ',['ㅆ','ㅇㅓㅇㅛ'],'ㅇㅛ']
]

EXC_4_deal_2 = [
    
    ['ㄴㅣ',['ㅆ','ㅇㅓㅇㅛ'],'ㅇㅛ']
]

EXC_4_deal_3 = [
    
    ['ㄴㅣ',['ㅆ','ㅇㅓㅇㅛ'],'ㅇㅛ']
]

EXC_4_deal_4 = [
    
    ['ㄱㅔ','ㄱㅔㅇㅛ','ㄱㅔ'],
    ['ㄷㅔ','ㄷㅔㅇㅛ','ㄷㅔ'],
    ['ㄱㅓㄹ','ㄱㅓㄹㅇㅛ','ㄱㅓㄹ']
]

#도록은 special

#### End of low -> high Dictionary####

#### Common Dictionary ####
#Yes or No
IC =[
    [['ㄴ','ㅔ'],['ㅇ','ㅡ','ㅇ']],
    [['ㅇ','ㅏ','ㄴ','ㅣ','ㅇ','ㅛ'],['ㅇ','ㅏ','ㄴ','ㅣ']],
    [['ㄴ','ㅔ'],['ㄱ','ㅡ','ㄹ','ㅐ']],
]


#### End of Common Dictionary ####

#### Main Function ####
def unite(input, dict):
    for i in dict:
        input = re.sub(i[0],i[1],input)
    return input
    
## 자모 단위로 문장을 나누고 합칠 때 쓰는 class ##
class Jamodealer:
    jamo = []
    pp = ''
    #각 단어들을 받아와서 자모단위로 나눈다.
    def __init__(self,lis_word):
    
        self.jamo = []
        for i in lis_word:
            self.jamo.append(split_syllables(i))
    
    ##사전에서 변환된 자모단위로 분리된 문장을 합칠 때 쓰는 함수이다.     
    def make_one(self):
        #list 형태로 저장된 자모들의 집합을 하나의 string pp에 저장한다. 
        self.pp = ''
        for i in self.jamo:
             self.pp= self.pp+i
        ##종성과 종성을 합쳐야 하는 경우가 있다면 합친다.        
        self.pp = unite(self.pp, con_dict)
        
        #자모 단위의 string에서 자모 단위로 사전을 만들고 거기에 index를 부여한다.        
        chars = list(set(self.pp))
        char_to_ix = { ch:i for i,ch in enumerate(chars) }
        ix_to_char = { i:ch for i,ch in enumerate(chars) }
        
        #자모 단위로 분리되었던 문장을 다시 하나로 합친다.
        jamo_numbers = [char_to_ix[x] for x in self.pp]
        restored_jamo = ''.join([ix_to_char[x] for x in jamo_numbers])
        #합쳐진 문장을 return 한다.
        restored_text = join_jamos(restored_jamo)
        return restored_text
##End of Jamodealer class##

def tojamo(korean_word):
    r_lst = []
    for w in list(korean_word.strip()):
        ## 영어인 경우 구분해서 작성함. 
        if '가'<=w<='힣':
            ## 588개 마다 초성이 바뀜. 
            ch1 = (ord(w) - ord('가'))//588
            ## 중성은 총 28가지 종류
            ch2 = ((ord(w) - ord('가')) - (588*ch1)) // 28
            ch3 = (ord(w) - ord('가')) - (588*ch1) - 28*ch2
            r_lst.append([CHOSUNG_LIST[ch1], JUNGSUNG_LIST[ch2], JONGSUNG_LIST[ch3]])
        else:
            r_lst.append([w])
    return r_lst

#Use in makestrdict
#Convert list to String
def li2str(input):
    st = ""
    for i in input:
        st = st+i
    return st

##Used when make dictionaries
def makestrdict(input):
    result = []
    for i in input:
        bullet = []
        for j in range(len(i)):
            gre = li2str(i[j])
            bullet.append(gre)
        result.append(bullet)
    return result

### *추가한 사전에 대한 str 사전을 생성, mapping 시키는 부분* ###

# * 기존의 방식은 다음과 같다. 어말어미는 어말어미 변환 방식 코드를 참고하면 된다.
# - 앞으로  사전을 추가하고 싶으면 여기서 코드를 추가하고 수정하면 된다.
# - dictionary를 만들고 해당 형태소에 해당하는 dictionary를 추가하면 된다.
#   예를 들어, EP(선어말어미)에 대한 사전을 만들고 싶다면 EP_dict = makestrdict(EP)를 선언하고
#   Dict_list에 'EP'를 추가하면 된다. 그 후 Mapping이 되도록 Dict_map에도 생성한 EP_dict를 추가하면 된다.

#EP_dict = makestrdict(EP)
EF_dict = makestrdict(EF)
NP_dict = makestrdict(NP)
JX_dict = makestrdict(JX)
VX_dict = makestrdict(VX)
VV_dict = makestrdict(VV)
XR_dict = makestrdict(XR)
NNG_dict = makestrdict(NNG)
NNP_dict = makestrdict(NNP)
UNKNOWN_dict = makestrdict(UNKNOWN)

Dict_list=['NNP','NP','JX','VX','VV','XR','NNG', 'UNKNOWN']

Dict_map = [NNP_dict,NP_dict,JX_dict,VX_dict, VV_dict, XR_dict, NNG_dict, UNKNOWN_dict]

##현재의 어말어미 변환방식은 밑에 코드를 추가, 변경하면 된다. 
#high->low of string dictionary
EF_ONLY_dict = makestrdict(EF_ONLY)
EP_EF_dict = makestrdict(EP_EF)
VCP_EF_dict = makestrdict(VCP_EF)
A_EF_dict = makestrdict(A_EF)
XSA_EF_dict = makestrdict(XSA_EF)
XSV_EF_dict = makestrdict(XSV_EF)


#low->high of string dictionary
EF_ONLY_4S_dict = makestrdict(EF_ONLY_4S)
EF_ONLY_4C_dict = makestrdict(EF_ONLY_4C)
EF_AFTER_VCP_4_dict = makestrdict(EF_AFTER_VCP_4)
VCP_EF_4_dict = makestrdict(VCP_EF_4)
EF_AFTER_XSA_4_dict = makestrdict(EF_AFTER_XSA_4) ## add code_JM
VA_EF_4_dict = makestrdict(VA_EF_4)
VV_EF_4_dict = makestrdict(VV_EF_4)
VX_EF_4_dict = makestrdict(VX_EF_4)
XSV_EF_4_dict = makestrdict(XSV_EF_4)
NNB_JKO_4_dict = makestrdict(NNB_JKO_4)##add code_JH
EC_4_dict = makestrdict(EC_4)
VV_4_dict = makestrdict(VV_4)
XSA_EF_4_dict = makestrdict(XSA_EF_4)


##Use in Changer
def to2lists(input):
    lis_word = []
    lis_tag = []
    #data = han.pos(input,ntags=22,flatten=True, join=False)
    data = mec.pos(input)
    for i in data:
        lis_word.append(i[0])
        lis_tag.append(i[1])
    return lis_word, lis_tag

#add in 2021.09.26

#space의 index 찾는다.
#rememberSpace와 convertSpace는 문장의 space를 찾아 변환하고 그 위치에 다시 space를 넣기 위한 함수이다.
def rememberSpace(lis, input):
    
    rlis = []
    
    for i in range(len(lis)):
        if lis[i]==input:
            rlis.append(i)
            
    for i in range(len(rlis)):
        rlis[i] = rlis[i]-i      
    return rlis

def convertSpace(lis_space,lis_lis):
    
    rlis = []
    k=0
    for i in range(len(lis_lis)):
        
        if k in lis_space:
            rlis.append(i)
            
        k = k+len(lis_lis[i])
        
    #print(rlis)  
    return rlis

def union(lis, lis_lis):
    
    k = 0
    for i in lis:
        lis_lis.insert(i+k,' ')
        k = k+1
        
##특정 형태소를 찾는다. 종결어미(EF)를 찾아서 높임말 반말 변환 시 사용된다.         
def findTargetMorphology(input, word):
    
    i=0
    lis = []
    for j in range(len(input)):
        if word in input[j]:
            lis.append(j)
            
    return lis

def del_over(input):
    
    res = set(input) #집합set으로 변환
    result = list(res) #list로 변환
    return result

## Main Class ## 
class Changer(object):
    
    ## High -> Low ##    
    def high_low(self, stc):
        result = stc
        
        space_list = rememberSpace(stc,' ')
        
        lis_word, lis_tag = to2lists(result)
        space_location = convertSpace(space_list, lis_word)
        jam = Jamodealer(lis_word)
        lis = []
        key = -1
        for i in H_LIST:
            if i in lis_word[-2]:
                key = 1
        #if key>0:
        if key>0 or key<0:     
            for i in range(len(lis_tag)):
                res = jam.jamo[i]
                #for k in range(len(Dict_list)):
                dic = []
                if 'EF' in lis_tag[i]:
                    if 'EF' == lis_tag[i]:
                        dic = EF_ONLY_dict
                    elif 'EP' in lis_tag[i]:
                        dic = EP_EF_dict
                    elif 'VCP' in lis_tag[i]:
                        dic = VCP_EF_dict
                    elif 'XSA' in lis_tag[i]:
                        dic = XSA_EF_dict
                    #####add
                    elif 'XSV' in lis_tag[i]:
                        dic = XSV_EF_dict
                    else:
                        dic = A_EF_dict
                        
                #else Dict_list[k] in lis_tag[i]:
                else:
                    for k in range(len(Dict_list)):
                        if Dict_list[k] in lis_tag[i]:
                            dic = Dict_map[Dict_list.index(Dict_list[k])]
                            break
                    #res = jam.jamo[i]
                        
                flag = 0    
                for j in range(len(dic)):
                    if dic[j][0] in res:
                        flag = 1
                            
                    if self.isExcept(dic[j])==1:
                        #ind = self.indicator(i,jam.jamo,lis_tag,EXC_word, EXC_tags)
                        for q in range(len(EXC_deal_1)):
                            if dic[j][0]==EXC_deal_1[q][0]:
                                for r in range(1,len(EXC_deal_1[q])):
                                    if jam.jamo[i-1][-1]==EXC_deal_1[q][r][0]:
                                        res = re.sub(dic[j][0],EXC_deal_1[q][r][1],res)
                        #res = re.sub(dic[j][0],dic[j][ind],res)
                    elif self.isExcept(dic[j])==2:
                        for q in range(len(EXC_deal_2)):
                            if dic[j][0]==EXC_deal_2[q][0]:
                                if i+1 in space_location:
                                    res = re.sub(dic[j][0], EXC_deal_2[q][2], res)
                                else:
                                    res = re.sub(dic[j][0], EXC_deal_2[q][1], res)
                    elif self.isExcept(dic[j])==3:
                        #ind = self.indicator(i,jam.jamo,lis_tag,EXC_word, EXC_tags)
                        for q in range(len(EXC_deal_3)):
                            if dic[j][0]==EXC_deal_3[q][0]:
                                if jam.jamo[i-1][-1]==EXC_deal_3[q][1][0]:
                                    res = re.sub(dic[j][0],EXC_deal_3[q][1][1],res)
                                else:
                                    res = re.sub(dic[j][0],EXC_deal_3[q][2],res)
                        #res = re.sub(dic[j][0],dic[j][ind],res)
                    else:
                        #print('tt')
                        res = re.sub(dic[j][0],dic[j][1],res)
                            
                    if flag==1:
                        break
                            
                        #jam.jamo[i] = res
                lis.append(res)
            
            
            #print(jam.jamo[i])
        
        union(space_location, lis)
        jam.jamo = []
        for i in range(len(lis)):
            jam.jamo.append(lis[i])
            #print(lis[i])
        
        #union(space_location, jam.jamo)
        
        return jam.make_one()
    
    def low_high(self, stc):
        result = stc
        
        space_list = rememberSpace(stc, ' ')
        
        lis_word, lis_tag = to2lists(result)
        space_location = convertSpace(space_list, lis_word)
        jam = Jamodealer(lis_word)
        #print(jam.jamo)
        lis = []
        
        ## 종결어미만을 변경
        ind_trans_ef = findTargetMorphology(lis_tag, 'EF')
        ind_trans_jko = findTargetMorphology(lis_tag, 'JKO')
        ind_trans_ec = findTargetMorphology(lis_tag, 'EC')
        ind_trans_vv = findTargetMorphology(lis_tag, 'VV')
        
        ind_union = ind_trans_ef + ind_trans_jko + ind_trans_ec + ind_trans_vv
        ind_union = del_over(ind_union)
        
        for ind in ind_union:  #modified_2021.10.31
            key = -1

            for i in H_LIST:
                if i not in lis_word[ind]:
                    key = 1
                else:
                    key = -1
                if key==-1:
                    break
            if key >0:
                res = jam.jamo[ind]
                dic = []
                #EF
                if 'EF' in lis_tag[ind]:

                    if 'EF' == lis_tag[ind]:
                        if '+' in lis_tag[ind-1]:
                            dic = EF_ONLY_4C_dict
                        elif lis_tag[ind-1]=='VCP':
                            dic = EF_AFTER_VCP_4_dict
                        elif lis_tag[ind-1]=='XSA':
                            dic = EF_AFTER_XSA_4_dict
                        else:
                            dic = EF_ONLY_4S_dict
                            #print('aa')
                    elif 'EP' in lis_tag[ind]:
                        dic = EP_EF_dict
                    elif 'VCP' in lis_tag[ind]:
                        dic = VCP_EF_4_dict
                    elif 'VA' in lis_tag[ind]:
                        dic = VA_EF_4_dict
                    elif 'VV' in lis_tag[ind]:
                        dic = VV_EF_4_dict
                    elif 'VX' in lis_tag[ind]:
                        dic = VX_EF_4_dict
                    elif 'XSV' in lis_tag[ind]:
                        dic = XSV_EF_4_dict
                    elif 'XSA' in lis_tag[ind]:
                        dic = XSA_EF_4_dict
                    else:
                        dic = A_EF_dict
                #JKO      
                elif 'JKO' in lis_tag[ind]:
                    dic = NNB_JKO_4_dict
                #EC      
                elif 'EC' in lis_tag[ind]:
                    dic = EC_4_dict
                elif 'VV' in lis_tag[ind]:
                    dic = VV_4_dict
            
                ##added by JM
                flag = 0        
                for j in range(len(dic)):
                    if dic[j][0] in res:
                        flag = 1
                            
                    if self.isExcept(dic[j])==1:
                        #ind = self.indicator(i,jam.jamo,lis_tag,EXC_word, EXC_tags)
                        for q in range(len(EXC_4_deal_1)):
                            if dic[j][0]==EXC_4_deal_1[q][0]:
                                for r in range(1,len(EXC_4_deal_1[q])):
                                    if jam.jamo[ind-1][-1]==EXC_4_deal_1[q][r][0]:
                                        res = re.sub(dic[j][0],EXC_4_deal_1[q][r][1],res)
                        #res = re.sub(dic[j][0],dic[j][ind],res)
                    elif self.isExcept(dic[j])==2:
                        for q in range(len(EXC_4_deal_2)):
                            if dic[j][0]==EXC_4_deal_2[q][0]:
                                if ind+1 in space_location:
                                    res = re.sub(dic[j][0], EXC_4_deal_2[q][2], res)
                                else:
                                    res = re.sub(dic[j][0], EXC_4_deal_2[q][1], res)
                    elif self.isExcept(dic[j])==3:
                        #ind = self.indicator(i,jam.jamo,lis_tag,EXC_word, EXC_tags)
                        for q in range(len(EXC_4_deal_3)):
                            if dic[j][0]==EXC_4_deal_3[q][0]:
                                if jam.jamo[ind-1][-1]==EXC_4_deal_3[q][1][0]:
                                    res = re.sub(dic[j][0],EXC_4_deal_3[q][1][1],res)
                                else:
                                    res = re.sub(dic[j][0],EXC_4_deal_3[q][2],res)
                    elif self.isExcept(dic[j])==4:
                        #ind = self.indicator(i,jam.jamo,lis_tag,EXC_word, EXC_tags)
                        for q in range(len(EXC_4_deal_4)):
                            if dic[j][0]==EXC_4_deal_4[q][0]:
                                if ind ==len(lis_tag)-1:
                                    res = re.sub(dic[j][0],EXC_4_deal_4[q][1],res)
                                else:
                                    res = re.sub(dic[j][0],EXC_4_deal_4[q][2],res)
                    
                    else:
                        if len(dic[j])>2:
                            res = re.sub(dic[j][0],dic[j][1],res)
                        else:
                            res = re.sub(dic[j][0],dic[j][1],res)
                        if flag==1:
                            break
                jam.jamo[ind] = res


            
        
        
        
        union(space_location, jam.jamo)
        
        return jam.make_one()
            
          
    def isExcept(self, input):
        if input[1]=='special':
            return 1
        elif input[1]=='special-':
            return 2
        elif input[1]=='-special':
            return 3
        elif input[1]=='specialx':
            return 4
        else:
            return 0
        
    def indicator(self, ind, lis, tag, ex_word, exc_tags):
        re = 1
        for j in range(len(ex_word)):
            if exc_tags[j] in tag[ind+1]:
                for i in range(len(ex_word)):
                    if ex_word[i][0] in lis[ind+1]:
                       
                        re = ex_word[i][1]
                        break
                        
                        print(lis[ind-1])
        return re
        
        
#     def processText(self,stc):
#         result = stc
#         res = self.high_low(result)
#         #spacing = Spacing()
#         #res = spacing(res)
#         return res
#     ##change function name
#     def processText_convertHigh(self,stc):
#         result = stc
#         res = self.low_high(result)
#         return res
    def processText(self,stc):
        result = stc
        
        flag = 0
        if result[-1]=='\n':
            result = result.replace('\n','')
        if result[-1] =='?' or result[-1] =='.' or result[-1] =='!' or result[-1] =='\"':
            result = result
        else:
            result = result+'.'
            flag = 1
        
        res = self.high_low(result)
        
        if flag ==1:
            res = res[:-1]
        
        #spacing = Spacing()
        #res = spacing(res)
        return res
    
    def processText_convertHigh(self,stc):
        result = stc
        
        flag = 0
        if result[-1]=='\n':
            result = result.replace('\n','')
        if result[-1] =='?' or result[-1] =='.' or result[-1] =='!' or result[-1] =='\"':
            result = result
        else:
            result = result+'.'
            flag = 1
        
        res = self.low_high(result)
        
        if flag ==1:
            res = res[:-1]
        return res
    

In [13]:
## Test convertion
#txt = '전 그것이 맞다고 생각합니다.' 는다, 다 처리 완료
tx = '싫다고요. 몇 번을 말해요? 정말 위대합니다! 빨리 입습니다. 아쉽네요. 물건을 듭니다. 전주로 가자. 전 주로 라면에 파를 넣어 먹습니다. 전 그렇게 생각하지 않습니다. 생각이 듭니다. 들었습니다. 사랑해요 나는 너와 결혼할 거야.\
당신을 사랑합니다.'
x = mec.pos(tx, flatten=True, join=True)
print(x)
ch = Changer()
#tt = ch.processText(txt)
ttt = ch.processText(tx)
text_low = '뛰어나다. 손꼽힌다. 아니다. 펼친다. 감사드린다. 만든다.무엇이 문제이오? 뛰어나다. 해야 한다. 받는다. 크다. 아니다. 하겠는가?'

converted_txt = ch.processText_convertHigh(text_low)
print(ttt)
print(converted_txt)

['싫/VA', '다고요/EF', './SF', '몇/MM', '번/NNBC', '을/JKO', '말/NNG', '해요/XSV+EF', '?/SF', '정말/MAG', '위대/XR', '합니다/XSA+EF', '!/SF', '빨리/MAG', '입/VV', '습니다/EF', './SF', '아쉽/VA', '네요/EF', './SF', '물건/NNG', '을/JKO', '듭니다/UNKNOWN', './SF', '전/NP+JX', '주로/MAG', '가/VV', '자/EF', './SF', '전/NP+JX', '주로/MAG', '라면/NNG', '에/JKB', '파/NNG', '를/JKO', '넣/VV', '어/EC', '먹/VV', '습니다/EF', './SF', '전/NP+JX', '그렇게/MAG', '생각/NNG', '하/XSV', '지/EC', '않/VX', '습니다/EF', './SF', '생각/NNG', '이/JKS', '듭니다/UNKNOWN', './SF', '들/VV', '었/EP', '습니다/EF', './SF', '사랑/NNG', '해요/XSV+EF', '나/NP', '는/JX', '너/NP', '와/JKB', '결혼/NNG', '할/XSV+ETM', '거/NNB', '야/VCP+EF', './SF', '당신/NP', '을/JKO', '사랑/NNG', '합니다/XSV+EF', './SF']
싫다고. 몇 번을 말하자? 정말 위대하다! 빨리 입는다. 아쉽다. 물건을 든다. 전주로 가자. 난 주로 라면에 파를 넣어 먹는다. 난 그렇게 생각하지 않는다. 생각이 든다. 들었다. 사랑하자 나는 너와 결혼할 거야.당신을 사랑해.
뛰어나습니다. 손꼽힙니다. 아니습니다. 펴욜친다. 감사드립니다. 만든습니다.무엇이 문제이세요? 뛰어나습니다. 해야 한다. 받는습니다. 크습니다. 아니습니다. 하겠는가세요?


In [26]:
import pandas as pd
number = 1000
texts = pd.read_table('./data/hgu_clean.kr', sep ='\n')[:number]
texts = texts['26일이요?']
for i in range(0, number):
    print(texts[i])

27일은 괜찮나요?
중단시켜서 죄송해요.
중단시켜서 죄송해요.
아, 이런! 도저히 생각이 안 나는걸.
Joe's Café 라는 레스토랑에 가고 싶은데, 알고 계세요?
보다 나은 건강과 생활 회사의 글렌 토마스입니다.
스타워즈는 어때?
데이터 코디네이터나 데이터 매니저가 어떨까요?
화성에서 온 로봇이요! 좋아요! 특수 효과가 대단하대요.
팩스를 보내줘서 감사합니다.
11월30일 날짜로 팩스를 보내줘서 감사합니다.
자격을 취득하기 위해 공부 중이에요.
간호사 자격을 취득하기 위해 공부 중이에요.
국가시험을 향해 공부 중이에요.
공무원에 되기 위한 국가시험을 향해 공부 중이에요.
잘 됐어요. 그럼 이제 집에 가도 되나요?
죄송합니다. 그녀는 안에 없네요. 메모 남겨 드릴까요?
내려 줄래요.
올려 줄래요.
작은 목소리
스미스 선생님. 저 화장실에 가야겠는데요.
화장실이 정말 급해요!
그럼 총수입의 얼마나 되는 거죠?
0136521798이에요.
1,199달러예요?
100그램에 비타민C의 함유량은 레몬보다 높습니다.
100미터 더 가서 좌회전해.
105번이 그 근처에 서는 것 같은데, 확실하진 않아요.
낸시에게 물어 보세요. 낸시는 나보다 더 자주 그 버스를 이용하거든요.
105편의 탑승게이트는 변경되었습니까?
10분 정도 후에 다시 걸어주세요.
10분 후에 다시 걸죠.
10분 후에 와요.
10분정도 기다리실 수 있습니까?
10세 여자 아이의 선물로는 무엇이 좋을 것 같습니까?
10시 정각이에요.
10시 정각이에요.
10시쯤에 목욕해요.
10일에는 바쁘십니까?
11번가와 브로드웨이가 만나는 모퉁이로 가는 버스가 있나요?
123호실 좀 대 주세요?
123호실이오.
125쪽입니다.
12시부터 1시까지예요.
12시쯤에 자요.
12일 월요일에 뉴욕에서 필라델피아로 가는 표를 구하려고 하는데요.
12페이지 밖에 안 돼요. 인쇄하는데 오래 걸리지 않을 거예요.
14달러 47센트입니다. 정말 죄송합니다.
15분 쯤 뒤에 내 방으로 와 주실래요?
15세이기때문에 술은 마실 수 없습니다

KeyError: 999

In [27]:
## For test sample convertion

import pandas as pd
number = 1000
texts = pd.read_table('./data/hgu_clean.kr', sep ='\n')[:number]
texts = texts['26일이요?']

ch = Changer()

for i in range(0, number):
    tt = ch.processText_convertHigh(texts[i])
    print(tt)

27일은 괜찮나요?
중단시켜서 죄송해요.
중단시켜서 죄송해요.
아, 이런! 도저히 생각이 안 나는걸요.
Joe's Café 라는 레스토랑에 가고 싶은데요, 알고 계세요?
보다 나은 건강과 생활 회사의 글렌 토마스입니다.
스타워즈는 어때요?
데이터 코디네이터나 데이터 매니저가 어떨까요?
화성에서 온 로봇이요! 좋아요! 특수 효과가 대단하대요.
팩스를 보내줘서 감사합니다.
11월30일 날짜로 팩스를 보내줘서 감사합니다.
자격을 취득하기 위해 공부 중이에요.
간호사 자격을 취득하기 위해 공부 중이에요.
국가시험을 향해 공부 중이에요.
공무원에 되기 위한 국가시험을 향해 공부 중이에요.
잘 됐어요. 그럼 이제 집에 가도 되나요?
죄송합니다. 그녀는 안에 없네요. 메모 남겨 드릴까요?
내려 줄래요.
올려 줄래요.
작은 목소리
스미스 선생님. 저 화장실에 가야겠는데요.
화장실이 정말 급해요!
그럼 총수입의 얼마나 되는 거죠?
0136521798이에요.
1,199달러예요?
100그램에 비타민C의 함유량은 레몬보다 높습니다.
100미터 더 가서 좌회전해요.
105번이 그 근처에 서는 것 같은데요, 확실하진 않아요.
낸시에게 물어 보세요. 낸시는 나보다 더 자주 그 버스를 이용하거든요.
105편의 탑승게이트는 변경되었습니까?
10분 정도 후에 다시 걸어주세요.
10분 후에 다시 걸죠.
10분 후에 와요.
10분정도 기다리실 수 있습니까?
10세 여자 아이의 선물로는 무엇이 좋을 것 같습니까?
10시 정각이에요.
10시 정각이에요.
10시쯤에 목욕해요.
10일에는 바쁘십니까?
11번가와 브로드웨이가 만나는 모퉁이로 가는 버스가 있나요?
123호실 좀 대 주세요?
123호실이세요.
125쪽입니다.
12시부터 1시까지예요.
12시쯤에 자요.
12일 월요일에 뉴욕에서 필라델피아로 가는 표를 구하려고 하는데요.
12페이지 밖에 안 돼요. 인쇄하는데요 오래 걸리지 않을 거예요.
14달러 47센트입니다. 정말 죄송합니다.
15분 쯤 뒤에 내 방으로 와 주실래요?
15세이기때문에 술은 마실

고마워요. 출장 여행 중이세요?
고마워요. 회사측에서도 우리가 이런 방식으로 작업을 진행해야 할 거라는 데 동의해 주면 좋겠어요.
대단히 고맙습니다.
대단히 감사합니다.
고맙습니다.
고맙습니다. 이번 주 월요일에 사용하실 건가요?
고맙습니다. 정말 고맙습니다. 수고하세요.
고맙지만, 다른 분이 도와주고 있어요.
고민거린 금방 해결 될 거에요.
고민을 들어줘서 고마워요.
고민을 상담할 수 있는 사람이 없어요.
고상한 취미를 가지고 계시네요.
고생하고 있어요.
고추 같은 자극성 강한 것은 먹지 않도록.
고치는 비용은 얼마 입니까?
고혈압이에요.
곧 그곳으로 응급구조원들을 출동시키겠습니다.
곧 그렇게 할 거예요.
곧 그렇게 할 것 같아요.
곧 도착할 것입니다.
곧 또 짐을 꾸려야 하는 것 아닌가 모르겠어요.
골프를 꽤 잘 해요.
골프에 푹 빠졌어요.
공도 세답니다.
공부는 잘돼가고 있어요?
공부하고 있었어요.
공사중인 간선 도로를 운전해 가는데요 조그마한 돌이 날아와 유리가 깨졌어요.
공연 안내처는 어디입니까?
공연은 어땠어요?
공원에 벗꽃 구경을 보러 갈 거예요.
공중전화는 어디예요?
공중전화를 찾고 있습니다만.
공통적인 입장을 찾아보면서 시작해야 할 거예요.
공포 영화였어요.
공포는 싫어해요.
공학 학위 정도는 있어야지 설치할 수 있을 것 같네요.
공항 근처에 가격이 적당한 호텔을 알려 주세요.
공항 근처의 호텔까지 가 주세요?
공항까지 가장 간단하게요 갈 수 있는 방법을 가르쳐 줄 수 있어요?
공항에 도착해서 해도 되나요?
공항에서 호텔에 어떻게요 가죠?
과식하지 않게요 조심해요.
과식하지 않도록 하고 있어요.
과음하지 않게요 조심해요.
과일 좋아하십니까?
과일과 야채, 그리고 완전 곡물식을 먹어요. 붉은 색 육류는 많이 먹지 않아요.
과일은 매일 먹고 있습니다.
과일주스를 주세요?
관광안내소까지 갈 방법을 가르쳐 줄 수 있어요?
관광안내소를 찾고 있는데요.
관광안내소에 가는 길을 가르쳐 주세요.
관광지가 실려있는 지도를 갖고 싶은데요.
관련 기술 문제들을 우

NameError: name 'EXC_4_deal_1' is not defined

In [59]:
## Make Sample Data file ##

number = 1858355
texts = pd.read_table('./data/hgu_clean.kr', sep ='\n')

new_data = {
    
    '26일이요?':'26일이요?'
    
}

idx = 0

temp1 = texts[texts.index < idx]
temp2 = texts[texts.index>= idx]

texts = temp1.append(new_data,ignore_index=True).append(temp2, ignore_index=True)

texts = texts['26일이요?']

num = len(texts)

ch = Changer()
f = open('./hgu_clean.low.kr', 'w',encoding = 'utf-8')
for i in range(0,num):
    if texts[i][-1] !='?'and texts[i][-1] !='.' and texts[i][-1] !='!':
        texts[i] = texts[i]+'.'
    tt = ch.processText(texts[i])
    f.write(tt)
    f.write('\n')
f.close()

In [ ]:
## Make Data file ##

f = open('./data/aihub.low1.kr.shuf', 'w',encoding = 'utf-8') # write file
f0 = open('./data/aihub.low.kr.shuf', 'r', encoding = 'utf-8') # read file

flag = 0

while True:
    flag = 0
    line = f0.readline()
    if not line: break
    if line[-1]=='\n':
        line = line.replace('\n','')
    if line[-1] =='?' or line[-1] =='.' or line[-1] =='!' or line[-1] =='\"':
        line = line
    else:
        line = line+'.'
        flag = 1
    tt = ch.processText(line)
    if flag ==1:
        tt[-1] = ''
    f.write(tt)
    f.write('\n')
f0.close()
f.close()